# Chatbot en Langue Française

Ce code implémente un **chatbot en français** en utilisant un modèle d'encodeur-décodeur (Encoder-Decoder) basé sur des réseaux de neurones récurrents (LSTM). Le chatbot est capable de répondre à des questions en français en se basant sur un dataset de questions-réponses, avec une interface graphique pour interagir facilement.

### Fonctionnement du Code

1. **Chargement du Dataset** :
   - Le dataset "CATIE-AQ/frenchQA" est utilisé, contenant des questions-réponses en français.
   - Les données sont extraites et limitées à 5000 échantillons pour faciliter l'entraînement.

2. **Prétraitement des Données** :
   - Les questions et réponses sont tokenisées, c'est-à-dire transformées en séquences numériques. 
   - Les séquences sont ensuite alignées à la même longueur via du **padding** pour assurer une entrée uniforme dans le modèle.
   - Les réponses sont formatées avec des tokens spéciaux (`<START>`, `<END>`) pour indiquer les débuts et fins de phrases.

3. **Définition du Modèle** :
   - Le modèle suit une architecture **encodeur-décodeur** :
     - L'encodeur (basé sur LSTM) prend en entrée les questions et génère des états internes représentant l'information.
     - Le décodeur (également basé sur LSTM) génère les réponses à partir des états de l'encodeur et d'une entrée initiale.
   - L'entraînement est effectué avec une perte **categorical_crossentropy** et l'optimiseur RMSprop sur 100 epochs.

4. **Sauvegarde et Chargement des Modèles** :
   - Les modèles d'encodeur et décodeur sont sauvegardés après l'entraînement pour être utilisés dans l'interface de chat.
   - Ces modèles sont ensuite chargés pour répondre aux questions posées par l'utilisateur.

5. **Interface Graphique (GUI)** :
   - Le code inclut une interface utilisateur construite avec **Tkinter**, permettant d'interagir avec le chatbot.
   - L'utilisateur entre une question dans une boîte de texte, et le chatbot génère une réponse qui s'affiche dans la zone de dialogue.

### Structure du Modèle

- **Encodeur** : Prend les questions en entrée et extrait des représentations vectorielles.
- **Décodeur** : Génère des réponses basées sur les états internes de l'encodeur et les tokens de début/fin.
- **Entraînement** : Utilise la méthode de **teacher forcing** pour améliorer l'apprentissage des séquences cibles.

### Objectif

Ce code constitue un point de départ pour créer un **chatbot en langue française** basé sur des réseaux de neurones séquentiels. Il peut être adapté à divers autres datasets ou langues, en ajustant les données d'entrée.



## Chargement et Prétraitement des Données

In [35]:
import numpy as np
from tensorflow.keras import preprocessing, utils
from keras import layers, models, activations, optimizers
from datasets import load_dataset

# Chargement du dataset "CATIE-AQ/frenchQA" contenant des questions et réponses en français
dataset = load_dataset("CATIE-AQ/frenchQA")

# Affichage d'un exemple de données du dataset pour observer la structure des données
print(dataset['train'][0])  

# Séparation des données d'entraînement du dataset
train = dataset["train"]

# Extraction des questions et des réponses
questions = train["question"]
answ = train["answers"]

# Affichage du type des données pour une question et la taille des listes de questions et réponses
print(type(questions[1]))
print(len(questions))
print(len(answ))



Reusing dataset french_qa (C:\Users\user\.cache\huggingface\datasets\CATIE-AQ___french_qa\frenchQA\1.0.0\4e913172efd12edfd4088c875a8864a8315c40f517d8ec5c50e3dbdfdb268a3a)
100%|██████████| 3/3 [00:00<00:00,  6.65it/s]


{'id': '0', 'title': 'pragnakalp/squad_v2_french_translated', 'context': "Beyoncé Giselle Knowles-Carter (/ biːˈjɒnseɪ / bee-YON-say) (née le 4 septembre 1981) est une chanteuse, compositrice, productrice de disques et actrice américaine. Née et élevée à Houston, au Texas, elle a joué dans divers chant et danse enfant, et est devenu célèbre à la fin des années 1990 en tant que chanteuse du groupe de filles R&B Destiny's Child. Géré par son père, Mathew Knowles, le groupe est devenu l'un des groupes de filles les plus vendus au monde de tous les temps. a vu la sortie du premier album de Beyoncé, Dangerously in Love (2003), qui l'a établie en tant qu'artiste solo dans le monde entier, a remporté cinq Grammy Awards et a présenté les singles numéro un du Billboard Hot 100 Crazy in Love et Baby Boy.", 'question': 'Quand Beyonce a-t-elle commencé à devenir populaire ?', 'answers': {'text': ['à la fin des années 1990'], 'answer_start': [269]}}
<class 'str'>
200617
200617


In [36]:
# Limitation des données à 5000 échantillons pour éviter des temps de traitement trop longs
questions = questions[:1499]
answ = answ[:1499]

# Préparation des réponses : On joint les textes de réponse en une seule chaîne de caractères
answers = [" ".join(ans['text']) for ans in answ]

# Ajout d'un exemple de question et réponse "bonjour" à la liste pour vérification
answers.append("bonjour")
questions.append("bonjour")

# Tokenization et padding des questions
# Tokenizer transforme le texte en séquences d'entiers correspondant aux mots
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions)  # Apprentissage du dictionnaire de mots basé sur les questions
encoded_questions = tokenizer.texts_to_sequences(questions)  # Conversion des questions en séquences numériques

# Calcul de la longueur maximale d'une question pour uniformiser la taille des séquences
max_len_q = max(len(seq) for seq in encoded_questions)

# Application de padding (complètement des séquences avec des zéros) pour aligner les questions à la même longueur
padded_questions = preprocessing.sequence.pad_sequences(encoded_questions, maxlen=max_len_q, padding='pre')

# Extraction du dictionnaire des mots pour les questions (mots associés à des indices numériques)
question_word_dict = tokenizer.word_index

# Préparation des réponses avec des tokens spéciaux "<START>" et "<END>" pour le modèle séquentiel
decoder_answers = ['<START>' + str(ans) + ' <END>' for ans in answers]

# Re-apprentissage du tokenizer sur les réponses formatées avec les tokens spéciaux
tokenizer.fit_on_texts(decoder_answers)
encoded_answers = tokenizer.texts_to_sequences(decoder_answers)  # Conversion des réponses en séquences numériques

# Calcul de la longueur maximale d'une réponse
max_len_a = max(len(seq) for seq in encoded_answers)

# Application de padding sur les réponses pour aligner les longueurs des séquences
padded_answers = preprocessing.sequence.pad_sequences(encoded_answers, maxlen=max_len_a, padding='post')

# Extraction du dictionnaire des mots pour les réponses
answer_word_dict = tokenizer.word_index

# Préparation des données cibles du décodeur : Décalage des réponses pour servir de cible lors de l'entraînement
decoder_target_data = [seq[1:] for seq in encoded_answers]

# Application de padding pour les séquences cibles et transformation en encodage one-hot pour classification
decoder_target_data = preprocessing.sequence.pad_sequences(decoder_target_data, maxlen=max_len_a, padding='post')
decoder_target_data = utils.to_categorical(np.array(decoder_target_data), len(answer_word_dict) + 1)

# Définition du modèle d'encodeur-décodeur
# Calcul du nombre de tokens uniques (mots) pour les questions et les réponses
question_tokens = len(question_word_dict) + 1
answer_tokens = len(answer_word_dict) + 1


# Modélisation et Compilation

In [37]:
# Encoder
encoder_inputs = layers.Input(shape=(None,))
encoder_embedding = layers.Embedding(input_dim=question_tokens, output_dim=200, mask_zero=True)(encoder_inputs)
encoder_outputs, state_h, state_c = layers.LSTM(200, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = layers.Input(shape=(None,))
decoder_embedding = layers.Embedding(input_dim=answer_tokens, output_dim=200, mask_zero=True)(decoder_inputs)
decoder_lstm = layers.LSTM(200, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = layers.Dense(answer_tokens, activation=activations.softmax)
decoder_outputs = decoder_dense(decoder_outputs)

# Compile and train the model
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=optimizers.RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_26      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_27      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_26        │ (None, None, 200) │    610,800 │ input_layer_26[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_26        │ (None, None)      │          0 │ input_layer_26[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_27        │ (None, None, 200) │    749,000 │ input_layer_27[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_26 (LSTM)      │ [(None, 200),     │    320,800 │ embedding_26[0][… │
│                     │ (None, 200),      │            │ not_equal_26[0][… │
│                     │ (None, 200)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_27 (LSTM)      │ [(None, None,     │    320,800 │ embedding_27[0][… │
│                     │ 200), (None,      │            │ lstm_26[0][1],    │
│                     │ 200), (None,      │            │ lstm_26[0][2]     │
│                     │ 200)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, None,      │    752,745 │ lstm_27[0][0]     │
│                     │ 3745)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,754,145 (10.51 MB)

 Trainable params: 2,754,145 (10.51 MB)

 Non-trainable params: 0 (0.00 B)

#  Entraînement du Modèle

In [38]:
# Train the model
model.fit([padded_questions, padded_answers], decoder_target_data, batch_size=64, epochs=50)


Epoch 1/50


d:\github\chat\chat1\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_169', 'keras_tensor_175']. Received: the structure of inputs=('*', '*')
  warnings.warn(


24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 435ms/step - accuracy: 0.7231 - loss: 7.7283
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 11s 436ms/step - accuracy: 0.8329 - loss: 4.6309
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 486ms/step - accuracy: 0.8369 - loss: 4.3901
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 480ms/step - accuracy: 0.3047 - loss: 4.1396
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 484ms/step - accuracy: 0.1246 - loss: 3.9864
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 504ms/step - accuracy: 0.1248 - loss: 3.8759
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 507ms/step - accuracy: 0.1249 - loss: 3.7917
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 501ms/step - accuracy: 0.1250 - loss: 3.8156
Epoch 9/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 485ms/step - accuracy: 0.1250 - loss: 3.7784
Epoch 10/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 484ms/step - accuracy: 0.1249 - loss: 3.7166
Epoch 11/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 491ms/step - accuracy: 0.1250 - loss: 3.6856
Epoch 12/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 490ms/ste

# Sauvegarde et Chargement des Modèles

In [39]:
# Save the models
def create_encoder_decoder_models():
    encoder_model = models.Model(encoder_inputs, encoder_states)
    decoder_input_state_h = layers.Input(shape=(200,))
    decoder_input_state_c = layers.Input(shape=(200,))
    decoder_input_states = [decoder_input_state_h, decoder_input_state_c]
    decoder_lstm_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding, initial_state=decoder_input_states)
    decoder_output_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_lstm_outputs)
    decoder_model = models.Model(
        [decoder_inputs] + decoder_input_states, [decoder_outputs] + decoder_output_states)
    return encoder_model, decoder_model

enc_model, dec_model = create_encoder_decoder_models()
enc_model.save('model/encoder_model_batsh_8.keras')
dec_model.save('model/decoder_model_batsh_8.keras')


#  Interface Utilisateur (GUI)

In [41]:
from tkinter import *
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import tensorflow.keras as k
import string

# Load pre-trained models
enc_model = k.models.load_model('model/encoder_model_batsh_8.keras', compile=False)
dec_model = k.models.load_model('model/decoder_model_batsh_8.keras', compile=False)

# Helper function to convert question to tokens
def question_to_tokens(question, max_len, question_word_dict):
    question = question.lower().translate(str.maketrans('', '', string.punctuation)).split()
    tokens = [question_word_dict.get(word, 0) for word in question]
    return pad_sequences([tokens], maxlen=max_len, padding='post')

# Function to generate responses from chatbot
def getResponse(question, enc_model, dec_model, max_len_q, answer_word_dict):
    states_values = enc_model.predict(question_to_tokens(question, max_len_q, question_word_dict))
    token = np.zeros((1, 1))
    token[0, 0] = answer_word_dict['start']
    
    stop_condition = False
    chatbot_answer = ''
    
    while not stop_condition:
        dec_output, h, c = dec_model.predict([token] + states_values)
        index = np.argmax(dec_output[0, 0, :])
        word = list(answer_word_dict.keys())[index - 1]
        
        # Stop if the word is 'end' but don't add 'end' to the answer
        if word == 'end' or len(chatbot_answer.split()) > max_len_a:
            stop_condition = True
        else:
            chatbot_answer += ' ' + word
            
        token[0, 0] = index
        states_values = [h, c]
    
    return chatbot_answer.strip()

# Chatbot GUI using Tkinter
def chatting():
    sen = entryBox.get('1.0', END).strip()
    entryBox.delete('1.0', END)
    chatArea.config(state=NORMAL)
    chatArea.insert(END, "You: " + sen + "\n\n")
    res = getResponse(sen, enc_model, dec_model, max_len_q, answer_word_dict)
    chatArea.insert(END, "Bot: " + res + "\n\n")
    chatArea.yview(END)
    chatArea.config(state=DISABLED)

# Initialize GUI
gui = Tk()
gui.title('ChatBot')
gui.geometry('400x500')
gui.resizable(width=False, height=False)

# Add GUI elements
chatArea = Text(gui, font=('Arial', 12))
chatArea.config(state=DISABLED)
scrollBar = Scrollbar(gui, command=chatArea.yview)
chatArea['yscrollcommand'] = scrollBar.set
chatButton = Button(gui, bd=0, command=chatting, text='Send', fg='white', font=('Arial', 18, 'bold'), bg='darkorange')
entryBox = Text(gui, font=('Arial', 12), height=2)

# Place components in the window
chatArea.place(x=6, y=6, height=380, width=380)
scrollBar.place(x=380, y=6, height=380, width=20)
chatButton.place(x=6, y=400, height=90)
entryBox.place(x=110, y=400, height=90, width=280)

# Run the GUI loop
gui.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [ ]:
# import numpy as np
# from tensorflow.keras import preprocessing, utils
# from keras import layers, models, activations, optimizers
# from datasets import load_dataset

# # Charger le dataset
# dataset = load_dataset("CATIE-AQ/frenchQA")
# train = dataset["train"]
# questions = train["question"]
# answ = train["answers"]

# # Limiter la taille des données pour l'exemple
# questions = questions[:3000]
# answ = answ[:3000]

# # Preprocessing des réponses
# answers = [" ".join(ans['text']) for ans in answ]
# answers = ['<START> ' + ans + ' <END>' for ans in answers]

# # Tokenisation et padding des questions
# tokenizer = preprocessing.text.Tokenizer()
# tokenizer.fit_on_texts(questions)
# encoded_questions = tokenizer.texts_to_sequences(questions)
# max_len_q = max(len(seq) for seq in encoded_questions)
# padded_questions = preprocessing.sequence.pad_sequences(encoded_questions, maxlen=max_len_q, padding='pre')
# question_word_dict = tokenizer.word_index

# # Tokenisation et padding des réponses
# tokenizer.fit_on_texts(answers)
# encoded_answers = tokenizer.texts_to_sequences(answers)
# max_len_a = max(len(seq) for seq in encoded_answers)
# padded_answers = preprocessing.sequence.pad_sequences(encoded_answers, maxlen=max_len_a, padding='post')
# answer_word_dict = tokenizer.word_index

# # Préparer les données cibles pour le décodeur
# decoder_target_data = [seq[1:] for seq in encoded_answers]
# decoder_target_data = preprocessing.sequence.pad_sequences(decoder_target_data, maxlen=max_len_a, padding='post')
# decoder_target_data = utils.to_categorical(np.array(decoder_target_data), len(answer_word_dict) + 1)

# # Taille des vocabulaires
# question_tokens = len(question_word_dict) + 1
# answer_tokens = len(answer_word_dict) + 1

# # Encoder
# encoder_inputs = layers.Input(shape=(None,))
# encoder_embedding = layers.Embedding(input_dim=question_tokens, output_dim=128, mask_zero=True)(encoder_inputs)
# encoder_outputs, state_h, state_c = layers.LSTM(128, return_state=True, return_sequences=True)(encoder_embedding)
# encoder_states = [state_h, state_c]

# # Decoder avec Attention
# decoder_inputs = layers.Input(shape=(None,))
# decoder_embedding = layers.Embedding(input_dim=answer_tokens, output_dim=128, mask_zero=True)(decoder_inputs)
# decoder_lstm = layers.LSTM(128, return_sequences=True, return_state=True)
# decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# # Transformer les sorties de l'encodeur pour s'aligner sur les sorties du décodeur
# encoder_dense = layers.Dense(128)(encoder_outputs)

# # Appliquer l'attention après la transformation
# attention = layers.Attention()([decoder_outputs, encoder_dense])
# decoder_concat_input = layers.Concatenate(axis=-1)([decoder_outputs, attention])

# # Couche de sortie
# decoder_dense = layers.Dense(answer_tokens, activation=activations.softmax)
# decoder_outputs = decoder_dense(decoder_concat_input)

# # Compilation du modèle
# model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

# # Entraîner le modèle
# model.fit([padded_questions, padded_answers], decoder_target_data, batch_size=32, epochs=50)

# # Fonction de sauvegarde des modèles encodeur/décodeur
# def create_encoder_decoder_models():
#     encoder_model = models.Model(encoder_inputs, encoder_states)
    
#     decoder_state_input_h = layers.Input(shape=(128,))
#     decoder_state_input_c = layers.Input(shape=(128,))
#     decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]
    
#     dec_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_state_inputs)
#     attention_inf = layers.Attention()([dec_outputs, encoder_dense])
#     decoder_concat_inf = layers.Concatenate(axis=-1)([dec_outputs, attention_inf])
#     dec_outputs = decoder_dense(decoder_concat_inf)
    
#     decoder_model = models.Model(
#         [decoder_inputs] + decoder_state_inputs, [dec_outputs] + [state_h, state_c]
#     )
    
#     return encoder_model, decoder_model

# # Sauvegarder les modèles
# enc_model, dec_model = create_encoder_decoder_models()
# enc_model.save('model/encoder_model.keras')
# dec_model.save('model/decoder_model.keras')
